# Pandas 데이터 구조 개요
`Series`, `Index`, and `DataFrame` classes.

## About the Data
In this notebook, we will be working with 5 rows from the earthquake data collected over September 18, 2018 - October 13, 2018 (obtained from the US Geological Survey (USGS) using the [USGS API](https://earthquake.usgs.gov/fdsnws/event/1/))

## Working with NumPy Arrays
`numpy` 배열로 csv 데이터를 불러온다.

In [6]:
import numpy as np

data = np.genfromtxt(
    'data/example_data.csv', delimiter=';', 
    names=True, dtype=None, encoding='UTF'
)
data

array([('2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia', 'mww', 6.7, 'green', 1),
       ('2018-10-13 04:34:15.580', '25km E of Bitung, Indonesia', 'mww', 5.2, 'green', 0),
       ('2018-10-13 00:13:46.220', '42km WNW of Sola, Vanuatu', 'mww', 5.7, 'green', 0),
       ('2018-10-12 21:09:49.240', '13km E of Nueva Concepcion, Guatemala', 'mww', 5.7, 'green', 0),
       ('2018-10-12 02:52:03.620', '128km SE of Kimbe, Papua New Guinea', 'mww', 5.6, 'green', 1)],
      dtype=[('time', '<U23'), ('place', '<U37'), ('magType', '<U3'), ('mag', '<f8'), ('alert', '<U5'), ('tsunami', '<i8')])

`shape` 

In [7]:
data.shape

(5,)

`dtype`

In [8]:
data.dtype

dtype([('time', '<U23'), ('place', '<U37'), ('magType', '<U3'), ('mag', '<f8'), ('alert', '<U5'), ('tsunami', '<i8')])

배열의 각 항목은 CSV 파일의 행이다. 데이터를 읽을 때, 다양한 유형의 데이터를 저장하는 데 사용되는 `numpy.void` 객체 배열이 만들어졌다. 행마다 다른 데이터 타입을 갖는 값들이 저장되기 때문.

timeit : 매직커맨드(앞에 %%) -> 이 계산이 시간이 얼마나 걸리는지 알려줌

In [9]:
%%timeit
max([row[3] for row in data])

2.31 µs ± 3.63 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


**[딕셔너리 컴프리헨션 dictionary comprehension](https://www.python.org/dev/peps/pep-0274/)**

In [10]:
array_dict = {
    col: np.array([row[i] for row in data])
    for i, col in enumerate(data.dtype.names) #행마다 i와 row를 받아와서 열의 이름들이 키(key). row[i] 넘파이 배열이 값으로 들어간다.
}
array_dict

{'time': array(['2018-10-13 11:10:23.560', '2018-10-13 04:34:15.580',
        '2018-10-13 00:13:46.220', '2018-10-12 21:09:49.240',
        '2018-10-12 02:52:03.620'], dtype='<U23'),
 'place': array(['262km NW of Ozernovskiy, Russia', '25km E of Bitung, Indonesia',
        '42km WNW of Sola, Vanuatu',
        '13km E of Nueva Concepcion, Guatemala',
        '128km SE of Kimbe, Papua New Guinea'], dtype='<U37'),
 'magType': array(['mww', 'mww', 'mww', 'mww', 'mww'], dtype='<U3'),
 'mag': array([6.7, 5.2, 5.7, 5.7, 5.6]),
 'alert': array(['green', 'green', 'green', 'green', 'green'], dtype='<U5'),
 'tsunami': array([1, 0, 0, 0, 1])}

최대 진도를 찾을 때, `mag` key와 `max()` 함수를 사용.

In [11]:
%%timeit
array_dict['mag'].max()

1.08 µs ± 3.52 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


하지만, 최대 진도값을 찾는 것에서 더 나아가서 해당 row를 다 출력하려면 

**최대 진도를 갖는 인덱스를 뽑아 각 행에 적용한다.**

(for문으로 각 행을 value로 받아온다.)

In [12]:
np.array([
    value[array_dict['mag'].argmax()] #mag 컬럼이 최대값인 인덱스 찾아서 값 찾기
    for key, value in array_dict.items()
])

array(['2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia',
       'mww', '6.7', 'green', '1'], dtype='<U32')

## `Series`
시리즈 클래스는 Numpy 배열과 마찬가지로 단일 유형의 배열을 위한 데이터구조.

In [13]:
import pandas as pd

place = pd.Series(array_dict['place'], name='place')
place

0          262km NW of Ozernovskiy, Russia
1              25km E of Bitung, Indonesia
2                42km WNW of Sola, Vanuatu
3    13km E of Nueva Concepcion, Guatemala
4      128km SE of Kimbe, Papua New Guinea
Name: place, dtype: object

`Series` 객체의 속성들. 

|Attribute | Returns |
| --- | --- |
| `name` | The name of the `Series` object |
| `dtype` | The data type of the `Series` object |
| `shape` | Dimensions of the `Series` object in a tuple of the form `(number of rows,)` |
| `index` | The `Index` object that is part of the `Series` object |
| `values` | The data in the `Series` object |


pandas 객체 대부분은 numpy 배열을 시영헤 내부 데이터를 표현한다. 그러나 일부 데이터 유형의 경우 pandas는 numpy 를 기반으로 한 pandas 배열을 생성한다. 이런 이유로 데이터 유형에 따라 결과 값은 `pandas.array`나 `numpy.array`가 될 수 있다. 따라서 특정 유형의 값을 얻으려면 valuyew대신 `array` 속성이나 `to_numpy()` 메서드를 사용하는 것이 좋다.

### Getting the name of the series

In [14]:
place.name

'place'

### Getting the data type
A `Series` object holds a single data type. Here it is `'O'` for object.

In [15]:
place.dtype

dtype('O')

### Getting the dimensions of the series

In [11]:
place.shape

(5,)

### Isolating the values from the series
This `Series` object is storing its values as a NumPy array:

In [16]:
place.values

array(['262km NW of Ozernovskiy, Russia', '25km E of Bitung, Indonesia',
       '42km WNW of Sola, Vanuatu',
       '13km E of Nueva Concepcion, Guatemala',
       '128km SE of Kimbe, Papua New Guinea'], dtype=object)

## `Index`
`Index` 클래스에서는 행 레이블을 통해 행을 선택할 수 있다.

In [23]:
place_index = place.index
place_index

RangeIndex(start=0, stop=5, step=1)

`values` 속성으로 실제 인덱스 데이터를 확인할 수 있다.

In [24]:
place_index.values

array([0, 1, 2, 3, 4])

`Index` objects의 속성들

|Attribute | Returns |
| --- | --- |
| `name` | The name of the `Index` object |
| `dtype` | The data type of the `Index` object |
| `shape` | Dimensions of the `Index` object |
| `values` | The data in the `Index` object |
| `is_unique` | Check if the `Index` object has all unique values |


In [25]:
place_index.dtype

dtype('int64')

In [17]:
place_index.shape

(5,)

값들의 고유성을 확인할 수 있다: **is_unique** (bool 속성)

In [26]:
place_index.is_unique

True

### 넘파이 연산

In [20]:
np.array([1, 1, 1]) + np.array([-1, 0, 1])

array([0, 1, 2])

### 판다스 연산

In [27]:
numbers = np.linspace(0, 10, num=5) #numpy array([0, 2.5, 5, 7.5, 10]), linear space
#위의 넘파이 배열을 인덱스를 다르게 해서 각각 x, y에 넣기
x = pd.Series(numbers) # index is [0, 1, 2, 3, 4]
y = pd.Series(numbers, index=pd.Index([1, 2, 3, 4, 5]))
x + y

0     NaN
1     2.5
2     7.5
3    12.5
4    17.5
5     NaN
dtype: float64

->공통적인 인덱스 1~4까지의 값만 x+y연산을 수행한다.

## `DataFrame`

->넘파이 표현을 DataFrame 객체로 만들 수 있다.

In [22]:
df = pd.DataFrame(array_dict) 
# this will also work with the first representation
# df = pd.DataFrame(data)

df

,time,place,magType,mag,alert,tsunami
0,2018-10-13 11:10:23.560,"262km NW of Ozernovskiy, Russia",mww,6.7,green,1
1,2018-10-13 04:34:15.580,"25km E of Bitung, Indonesia",mww,5.2,green,0
2,2018-10-13 00:13:46.220,"42km WNW of Sola, Vanuatu",mww,5.7,green,0
3,2018-10-12 21:09:49.240,"13km E of Nueva Concepcion, Guatemala",mww,5.7,green,0
4,2018-10-12 02:52:03.620,"128km SE of Kimbe, Papua New Guinea",mww,5.6,green,1


In [23]:
df2 = pd.DataFrame(data)
df2

,time,place,magType,mag,alert,tsunami
0,2018-10-13 11:10:23.560,"262km NW of Ozernovskiy, Russia",mww,6.7,green,1
1,2018-10-13 04:34:15.580,"25km E of Bitung, Indonesia",mww,5.2,green,0
2,2018-10-13 00:13:46.220,"42km WNW of Sola, Vanuatu",mww,5.7,green,0
3,2018-10-12 21:09:49.240,"13km E of Nueva Concepcion, Guatemala",mww,5.7,green,0
4,2018-10-12 02:52:03.620,"128km SE of Kimbe, Papua New Guinea",mww,5.6,green,1


In [21]:
df.dtypes

time        object
place       object
magType     object
mag        float64
alert       object
tsunami      int64
dtype: object

`values`속성으로는 각 행씩, 한 줄씩 출력된다.

In [22]:
df.values

array([['2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia',
        'mww', 6.7, 'green', 1],
       ['2018-10-13 04:34:15.580', '25km E of Bitung, Indonesia', 'mww',
        5.2, 'green', 0],
       ['2018-10-13 00:13:46.220', '42km WNW of Sola, Vanuatu', 'mww',
        5.7, 'green', 0],
       ['2018-10-12 21:09:49.240',
        '13km E of Nueva Concepcion, Guatemala', 'mww', 5.7, 'green', 0],
       ['2018-10-12 02:52:03.620', '128km SE of Kimbe, Papua New Guinea',
        'mww', 5.6, 'green', 1]], dtype=object)

`columns` 속성을 통해 열이름을 확인할 수 있고, 실제로 이 값은 `Index` 객체에도 저장된다.

In [23]:
df.columns

Index(['time', 'place', 'magType', 'mag', 'alert', 'tsunami'], dtype='object')

자주 쓰는 DataFrame의 속성들:

|Attribute | Returns |
| --- | --- |
| `dtypes` | The data types of each column |
| `shape` | Dimensions of the `DataFrame` object in a tuple of the form `(number of rows, number of columns)` |
| `index` | The `Index` object along the rows of the `DataFrame` object |
| `columns` | The name of the columns (as an `Index` object) |
| `values` | The data in the `DataFrame` object |
| `empty` | Check if the `DataFrame` object is empty |

The `Index` object along the rows of the dataframe can be accessed via the `index` attribute (just as with `Series` objects):

In [24]:
df.index

RangeIndex(start=0, stop=5, step=1)

In [24]:
df.shape

(5, 6)

df끼리도 산술연산이 가능하다, 숫자는 더해지고, 문자열은 concatenate 연접된다. 단, 인덱스의 수와 열의 수가 같아야 한다. dataframe의 모양이 같아야 한다.

In [26]:
df + df

,time,place,magType,mag,alert,tsunami
0,2018-10-13 11:10:23.5602018-10-13 11:10:23.560,"262km NW of Ozernovskiy, Russia262km NW of Oze...",mwwmww,13.4,greengreen,2
1,2018-10-13 04:34:15.5802018-10-13 04:34:15.580,"25km E of Bitung, Indonesia25km E of Bitung, I...",mwwmww,10.4,greengreen,0
2,2018-10-13 00:13:46.2202018-10-13 00:13:46.220,"42km WNW of Sola, Vanuatu42km WNW of Sola, Van...",mwwmww,11.4,greengreen,0
3,2018-10-12 21:09:49.2402018-10-12 21:09:49.240,"13km E of Nueva Concepcion, Guatemala13km E of...",mwwmww,11.4,greengreen,0
4,2018-10-12 02:52:03.6202018-10-12 02:52:03.620,"128km SE of Kimbe, Papua New Guinea128km SE of...",mwwmww,11.2,greengreen,2


<hr>
<div>
    <a href="../ch_01/introduction_to_data_analysis.ipynb">
        <button style="float: left;">&#8592; Chapter 1</button>
    </a>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>